In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

# Ustawienia ogólne
a, b = 0, 12
x_full = np.arange(a, b, 0.05)
y_full = 0.5 * np.cos(0.2 * x_full**2) + 0.5

epochs_list = [50, 100, 500]
samples_list = [20, 50, 100, 300, 1000]
net_configs = [
    [1], [4], [8], [20], [40], [20, 20]
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.makedirs("Wyniki", exist_ok=True)

# Dane do błędów MSE
mse_train_data = []
mse_test_data = []

# Definicja dynamicznej sieci
class DynamicNet(nn.Module):
    def __init__(self, layers):
        super(DynamicNet, self).__init__()
        modules = []
        input_size = 1
        for hidden_size in layers:
            modules.append(nn.Linear(input_size, hidden_size))
            modules.append(nn.ReLU())
            input_size = hidden_size
        modules.append(nn.Linear(input_size, 1))
        self.model = nn.Sequential(*modules)

    def forward(self, x):
        return self.model(x)

# Główna pętla eksperymentów
for epochs in epochs_list:
    for samples in samples_list:
        for config in net_configs:
            # === DANE UCZĄCE ===
            x_train = a + (b - a) * np.random.rand(samples, 1)
            y_train = 0.5 * np.cos(0.2 * x_train**2) + 0.5 + 0.1 * np.random.randn(samples, 1)

            x_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
            y_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

            # === DANE TESTOWE ===
            x_test = a + (b - a) * np.random.rand(300, 1)
            y_test = 0.5 * np.cos(0.2 * x_test**2) + 0.5
            x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
            y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

            # === TWORZENIE I UCZENIE SIECI ===
            net = DynamicNet(config).to(device)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(net.parameters(), lr=0.01)

            for epoch in range(epochs):
                net.train()
                optimizer.zero_grad()
                output = net(x_tensor)
                loss = criterion(output, y_tensor)
                loss.backward()
                optimizer.step()

            # === PREDYKCJA I BŁĘDY MSE ===
            net.eval()
            with torch.no_grad():
                y_pred_train = net(x_tensor)
                y_pred_test = net(x_test_tensor)
                mse_train = criterion(y_pred_train, y_tensor).item()
                mse_test = criterion(y_pred_test, y_test_tensor).item()

            mse_train_data.append([config, epochs, samples, mse_train])
            mse_test_data.append([config, epochs, samples, mse_test])

            # === WYKRESY APROKSYMACJI ===
            x_plot = torch.tensor(x_full.reshape(-1, 1), dtype=torch.float32).to(device)
            with torch.no_grad():
                y_pred_plot = net(x_plot).cpu().numpy()

            plt.figure(figsize=(8, 4))
            plt.plot(x_full, y_full, 'k-', label='Funkcja docelowa')
            plt.plot(x_train, y_train, 'rx', label='Dane uczące', markersize=4)
            plt.plot(x_full, y_pred_plot, 'b-', label='Aproksymacja sieci')
            plt.title(f'Neurony: {config}, Epoki: {epochs}, Próbki: {samples}\nMSE(train): {mse_train:.4f} | MSE(test): {mse_test:.4f}')
            plt.xlabel('x'); plt.ylabel('y'); plt.legend(); plt.grid(True)
            filename = f"Wyniki/wynik_N{'-'.join(map(str, config))}_E{epochs}_D{samples}.png"
            plt.savefig(filename)
            plt.close()

# === WYKRESY MSE (zbiorcze) ===
def plot_mse(data, title, filename):
    plt.figure(figsize=(10, 6))
    for config in net_configs:
        label = f"N={config}"
        mse_vals = [row[3] for row in data if row[0] == config and row[1] == 100]  # epoki=100
        x_vals = [row[2] for row in data if row[0] == config and row[1] == 100]
        plt.plot(x_vals, mse_vals, marker='o', label=label)
    plt.title(title)
    plt.xlabel('Liczba próbek uczących')
    plt.ylabel('MSE')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"Wyniki/{filename}")
    plt.close()

plot_mse(mse_train_data, "Błąd MSE - dane uczące (epoki=100)", "mse_train_ep100.png")
plot_mse(mse_test_data, "Błąd MSE - dane testowe (epoki=100)", "mse_test_ep100.png")

# === OPCJONALNIE: zapis błędów do CSV ===
with open("Wyniki/mse_train.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Neurony", "Epoki", "Dane", "MSE_Train"])
    for row in mse_train_data:
        writer.writerow(["-".join(map(str, row[0])), row[1], row[2], row[3]])

with open("Wyniki/mse_test.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Neurony", "Epoki", "Dane", "MSE_Test"])
    for row in mse_test_data:
        writer.writerow(["-".join(map(str, row[0])), row[1], row[2], row[3]])
